# Nilearn

If you're working on NeuroImaging data, you should check another Python library, **Nilearn**, that is design for  fast and easy statistical learning on NeuroImaging data. It leverages the scikit-learn Python toolbox for multivariate statistics with applications such as predictive modeling, classification, decoding, or connectivity analysis. Use their [website](http://nilearn.github.io/) to find out more.

As an example of how to use Nilearn, we will use the Haxby 2001 study on a face vs cat discrimination task in a mask of the ventral stream. This part is based on a [Nilearn tutorial](https://nilearn.github.io/auto_examples/plot_decoding_tutorial.html).

**Note** that first time you fetch the data, it can take a few minutes.

## Downloading data

In [ ]:
from nilearn import datasets

# By default 2nd subject will be fetched
haxby_dataset = datasets.fetch_haxby()

We can access anatomical, functional and mask data. And in addition we have true labels.

In [ ]:
func_file = haxby_dataset.func[0]
mask_file = haxby_dataset.mask_vt[0]
anat_file = haxby_dataset.anat[0]
labels_file = haxby_dataset.session_target[0]

let's get some info about ``bold`` file:

In [ ]:
!nib-ls $func_file

## Convert the fMRI volume’s to a data matrix using masks

We need our functional data in a 2D, sample-by-voxel matrix. To get that, we'll select a set of voxels in Ventral Temporal cortex defined by mask from the Haxby study:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from nilearn import plotting
plotting.plot_roi(mask_file, anat_file, cmap='Paired', dim=-.5)

Now we will create masker using the ``NiftiMasker``. ``NiftiMasker`` is an object that applies a mask to a dataset and returns the masked voxels as a vector at each time point. Here we use ``standardizing=True`` the time-series are centered and normed.

In [ ]:
from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_file, standardize=True)

# We give the masker a filename and retrieve a 2D array ready
# for machine learning with scikit-learn
fmri_masked = masker.fit_transform(func_file)

``fmr_mask`` is a Numpy array and its shape corresponds to the number of time-points times the number of voxels in the mask.

In [ ]:
print(fmri_masked.shape)

To recover the original data shape (giving us a masked and z-scored BOLD series), you can use ``masker.inverse_transform``.

##  Load the behavioral labels and choosing conditions

The ``label_file`` is CSV file, we can read it using Numpy:

In [ ]:
labels =  np.recfromcsv(labels_file, delimiter=" ")
labels

It's an array that have ``labels`` that gives information about condition and chunks represents a run number. We will use conditions: 

In [ ]:
conditions = labels['labels']
np.unique(conditions)

We see that there are 9 different conditions, but we will use faces and cats. Let's create another mask (this time masking the time points) that we will apply to our ``fmri_mask``

In [ ]:
condition_mask = np.logical_or(conditions == b'face', conditions == b'cat')
conditions_2lb = conditions[condition_mask]
fmri_masked_2lb = fmri_masked[condition_mask]
print(fmri_masked_2lb.shape)

## Decoding with an SVM

Now we will use a learning algorithm from scikit-learn to apply to our neuroImaging data. We will use a Support Vector Classification, with a linear kernel.

In [ ]:
from sklearn.svm import SVC
svc = SVC(kernel='linear')

Let's split our data and fit our model using the training set:

In [ ]:
from sklearn.cross_validation import train_test_split
fmri_tr, fmri_ts, cond_tr, cond_ts = train_test_split(fmri_masked_2lb, conditions_2lb)
svc.fit(fmri_tr, cond_tr)

And we can check the score for the testing set:

In [ ]:
svc.score(fmri_ts, cond_ts)

### Exercise 6

Validate the model using ``cross_val_score``. Try different kernels for SVM (you can read more [here](http://scikit-learn.org/stable/modules/svm.html)).

### Exercise 7

Check if ``KNeighborsClassifier`` would work for this dataset. Validate the model in the same way as SVC.

We can check weights assigned to the features by the model:

In [ ]:
coef = svc.coef_
print(coef)

Our array should have the same size as the VT mask:

In [ ]:
coef.shape

We need to turn it back into an original Nifti image, in essence, “inverting” what the NiftiMasker has done. For this, we can call ``inverse_transform`` on the ``NiftiMasker``:

In [ ]:
coef_img = masker.inverse_transform(coef)
print(coef_img)

If we need, we can save the image:

In [ ]:
coef_img.to_filename('haxby_svc_weights.nii.gz')

## Plotting the SVM weights

Now, lets plot our weights on top of the anatomical image:

In [ ]:
from nilearn.plotting import plot_stat_map

plot_stat_map(coef_img, anat_file,
              title="SVM weights", display_mode="yx")

Now you can see which area in VT cortex are important to distinguish between the two conditions according to our model.

### Exercise 8

Try to run model using all conditions (except rest state). This is multiclass classification, try one-vs-all and one-vs-one strategies (can read more [here](https://en.wikipedia.org/wiki/Multiclass_classification)), which one should be faster? 
Does the new model has as high score as the one with two conditions only? Which conditions is the easiest to identify by the model and which one is the hardest?